In [1]:
! pip install rdkit

In [2]:
import pandas as pd
import numpy as np

from rdkit import Chem

In [3]:
df_train = pd.read_csv("train.csv")

In [4]:
df_train["smiles_to_delete"] = df_train["smiles"]

1 - We create one column per element

In [5]:
#One column per element 
#No He, Li, Be, Ne, Ar, K, Sc, V, Mn, Fe, Ni, Cu, Kr, Rb, Sr, Y, Zr, Nb, Mo, Tc, Ru, Rh, Pd and after in the dataset
df_train["Number of H"] = 0
df_train["Number of B"] = 0
df_train["Number of C"] = 0
df_train["Number of N"] = 0
df_train["Number of O"] = 0
df_train["Number of F"] = 0
df_train["Number of Si"] = 0
df_train["Number of P"] = 0
df_train["Number of S"] = 0
df_train["Number of Cl"] = 0
df_train["Number of Br"] = 0
df_train["Number of I"] = 0

#In the dataset but very few amount (so drop these lines):
#Na (2 lines) Mg (1 line) Al (1 line) Ca (1 line) Ti (1 line) Cr (1 line) [Co] (1 line) Zn (1 line) Ge (1 line) As (4 lines)
#Se (7 lines) Ag (1 line) [Hg] (2 lines)

2 - We remove in "smiles_to_delete", the elements that appears less than 10 times in the dataset:

In [6]:
element_to_remove = ["Na", "Mg", "Al", "Ca", "Ti", "Cr", "[Co]", "Zn", "Ge", "As", "Se", "Ag", "[Hg]"]

# Liste pour stocker les indices des lignes à supprimer
indices_to_remove = []

for index, row in df_train.iterrows():
    for element in element_to_remove:
        if element in row["smiles_to_delete"]:
            indices_to_remove.append(index)  # Ajoutez l'indice de la ligne à supprimer

# Supprimez les lignes correspondantes
df_train.drop(indices_to_remove, inplace=True)

# Réinitialisez les index du DataFrame
df_train.reset_index(drop=True, inplace=True)

3 - We remove the special characters

In [7]:
char_to_remove = ["(", ")", "=", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "@", "[", "]", "-", "+", "H", "/", "#", ".", "\\", "%", " "] 
                  #We remove H car it is not suppose to be in smile

for index, row in df_train.iterrows():
    cleaned_smiles = ''.join([char for char in row["smiles_to_delete"] if char not in char_to_remove])
    df_train.loc[index, "smiles_to_delete"] = cleaned_smiles
                  

In [8]:
df_train

,name,smiles,mpC,smiles_to_delete,Number of H,Number of B,Number of C,Number of N,Number of O,Number of F,Number of Si,Number of P,Number of S,Number of Cl,Number of Br,Number of I
0,3-i-pr-5-mephenyl-n-me carbamate,O=C(Oc1cc(C)cc(c1)C(C)C)NC,87.0,OCOcccCcccCCCNC,0,0,0,0,0,0,0,0,0,0,0,0
1,b-2-furylacrylic acid,O=C(O)C=Cc1occc1,133.0,OCOCCcoccc,0,0,0,0,0,0,0,0,0,0,0,0
2,cyclacillin,OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C,182.5,OCOCNCOCNCOCNCCCCCCSCCC,0,0,0,0,0,0,0,0,0,0,0,0
3,"2,6-dihydroxy-4-methylpyridine",Cc1cc(nc(c1)O)O,197.0,CcccnccOO,0,0,0,0,0,0,0,0,0,0,0,0
4,dimethylnitramine,CN(C)[N+]([O-])=O,58.0,CNCNOO,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,"diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...",O=C(OCC)CC(CC(=O)OCC)(c1cc(C)c(O)c(C)c1)c1cc(C...,178.0,OCOCCCCCCOOCCcccCcOcCccccCcOcCc,0,0,0,0,0,0,0,0,0,0,0,0
19907,diethyl azelate,O=C(CCCCCCCC(=O)OCC)OCC,-18.5,OCCCCCCCCCOOCCOCC,0,0,0,0,0,0,0,0,0,0,0,0
19908,4-methyl-3-nitrophenol,Cc1ccc(cc1[N+](=O)[O-])O,79.0,CccccccNOOO,0,0,0,0,0,0,0,0,0,0,0,0
19909,(s)-3-(boc-amino)piperidine,CC(C)(C)OC(=O)N[C@H]1CCCNC1,125.0,CCCCOCONCCCCNC,0,0,0,0,0,0,0,0,0,0,0,0


4- We remove the important elements from smiles_to_delete and add the count in the column

In [9]:
element_to_count = ["Br", "B", "Cl", "C", "N", "O", "F", "Si", "P", "S", "I"]  # L'ordre est important !!!

for index, row in df_train.iterrows():
    for element in element_to_count:
        count = row["smiles_to_delete"].count(element.lower()) + row["smiles_to_delete"].count(element)
        df_train.loc[index, "Number of " + element] = count  # Ajouter la colonne pour le nombre d'occurrences
        row["smiles_to_delete"] = row["smiles_to_delete"].replace(element.lower(), "").replace(element, "")  # Supprimer l'élément de la chaîne (en ignorant la casse)
    # Mettre à jour la colonne "smiles_to_delete" dans le DataFrame
    df_train.loc[index, "smiles_to_delete"] = row["smiles_to_delete"]

In [10]:
df_train

,name,smiles,mpC,smiles_to_delete,Number of H,Number of B,Number of C,Number of N,Number of O,Number of F,Number of Si,Number of P,Number of S,Number of Cl,Number of Br,Number of I
0,3-i-pr-5-mephenyl-n-me carbamate,O=C(Oc1cc(C)cc(c1)C(C)C)NC,87.0,,0,0,12,1,2,0,0,0,0,0,0,0
1,b-2-furylacrylic acid,O=C(O)C=Cc1occc1,133.0,,0,0,7,0,3,0,0,0,0,0,0,0
2,cyclacillin,OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C,182.5,,0,0,15,3,4,0,0,0,1,0,0,0
3,"2,6-dihydroxy-4-methylpyridine",Cc1cc(nc(c1)O)O,197.0,,0,0,6,1,2,0,0,0,0,0,0,0
4,dimethylnitramine,CN(C)[N+]([O-])=O,58.0,,0,0,2,2,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,"diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...",O=C(OCC)CC(CC(=O)OCC)(c1cc(C)c(O)c(C)c1)c1cc(C...,178.0,,0,0,25,0,6,0,0,0,0,0,0,0
19907,diethyl azelate,O=C(CCCCCCCC(=O)OCC)OCC,-18.5,,0,0,13,0,4,0,0,0,0,0,0,0
19908,4-methyl-3-nitrophenol,Cc1ccc(cc1[N+](=O)[O-])O,79.0,,0,0,7,1,3,0,0,0,0,0,0,0
19909,(s)-3-(boc-amino)piperidine,CC(C)(C)OC(=O)N[C@H]1CCCNC1,125.0,,0,0,10,2,2,0,0,0,0,0,0,0


In [11]:
df_train["Number of B"].value_counts()

0     19697
1       211
2         1
3         1
10        1
Name: Number of B, dtype: int64

In [12]:
rows_with_value_10 = df_train[df_train["Number of B"] == 10]

# Afficher les lignes correspondantes
print(rows_with_value_10)

             name      smiles    mpC smiles_to_delete  Number of H  \
14788  decaborane  BBBBBBBBBB  101.0                             0   

       Number of B  Number of C  Number of N  Number of O  Number of F  \
14788           10            0            0            0            0   

       Number of Si  Number of P  Number of S  Number of Cl  Number of Br  \
14788             0            0            0             0             0   

       Number of I  
14788            0  


5-Let's suppose that we are not interest by the number of H for now

6- Let's count the number of insaturation

In [18]:
for index, row in df_train.iterrows():
    mol = Chem.MolFromSmiles(row["smiles"])

    # Vérifier si la conversion SMILES en molécule a réussi
    if mol is not None:
        # Obtenir le nombre d'atomes d'hydrogène dans la molécule
        nombre_non_hydrogenes = mol.GetNumHeavyAtoms()
        nombre_total_atomes = mol.GetNumAtoms()
        nombre_hydrogenes = nombre_total_atomes - nombre_non_hydrogenes
        df_train.loc[index, "Number of H"] = nombre_hydrogenes

[20:34:28] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[20:34:28] Explicit valence for atom # 8 N, 4, is greater than permitted
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 18
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[20:34:28] Explicit valence for atom # 20 C, 5, is greater than permitted
[20:34:28] Explicit valence for atom # 5 N, 4, is greater than permitted
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 6
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 17
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[20:34:28] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
[20:34:28] Explicit valence for atom # 6 N, 5, is greater than permitted
[20:34:28] Can't kekulize mol.  U

[20:34:32] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 7 8 9 10 11
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 33 34 35 36
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 18 19 20 21 22
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11 12 13 14 15
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 17
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 11 12 14 15
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 14
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10 11 12 13 14
[20:34:32] Can't kekulize mol.  Unkekulized atoms: 33 34 35 36 37
[

In [19]:
df_train

,name,smiles,mpC,smiles_to_delete,Number of H,Number of B,Number of C,Number of N,Number of O,Number of F,Number of Si,Number of P,Number of S,Number of Cl,Number of Br,Number of I
0,3-i-pr-5-mephenyl-n-me carbamate,O=C(Oc1cc(C)cc(c1)C(C)C)NC,87.0,,0,0,12,1,2,0,0,0,0,0,0,0
1,b-2-furylacrylic acid,O=C(O)C=Cc1occc1,133.0,,0,0,7,0,3,0,0,0,0,0,0,0
2,cyclacillin,OC(=O)C2N3C(=O)C(NC(=O)C1(N)CCCCC1)C3SC2(C)C,182.5,,0,0,15,3,4,0,0,0,1,0,0,0
3,"2,6-dihydroxy-4-methylpyridine",Cc1cc(nc(c1)O)O,197.0,,0,0,6,1,2,0,0,0,0,0,0,0
4,dimethylnitramine,CN(C)[N+]([O-])=O,58.0,,0,0,2,2,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,"diethyl 3,3-bis(4-hydroxy-3,5-dimethylphenyl)p...",O=C(OCC)CC(CC(=O)OCC)(c1cc(C)c(O)c(C)c1)c1cc(C...,178.0,,0,0,25,0,6,0,0,0,0,0,0,0
19907,diethyl azelate,O=C(CCCCCCCC(=O)OCC)OCC,-18.5,,0,0,13,0,4,0,0,0,0,0,0,0
19908,4-methyl-3-nitrophenol,Cc1ccc(cc1[N+](=O)[O-])O,79.0,,0,0,7,1,3,0,0,0,0,0,0,0
19909,(s)-3-(boc-amino)piperidine,CC(C)(C)OC(=O)N[C@H]1CCCNC1,125.0,,0,0,10,2,2,0,0,0,0,0,0,0


In [ ]:
voir : https://rdkit.org/docs/source/rdkit.Chem.html